In [1]:
import pandas as pd
import Bio
from Bio import SeqIO

In [4]:
kmar_gff = 'kmarx/kmarx_PASA2_noindels-newnums.gff3'
klac_gff = 'klact/klactis_PASA2_noindels.gff3'

In [5]:
kmar = pd.read_csv(kmar_gff, sep='\t', header=None, comment='#')
klac = pd.read_csv(klac_gff, sep='\t', header=None, comment='#')

gff = pd.concat([kmar, klac])
gff.head()

,0,1,2,3,4,5,6,7,8
0,chr39,YGOB/YGAP,gene,1721,3279,.,-,.,ID=KM020AG00100;Name=KM020AG00100
1,chr39,YGOB/YGAP,mRNA,1721,3279,.,-,.,ID=KM020AG00100;Parent=KM020AG00100;Name=KM020...
2,chr39,YGOB/YGAP,exon,1721,3279,.,-,.,ID=KM020AG00100.exon1;Parent=KM020AG00100
3,chr39,YGOB/YGAP,CDS,1721,3279,.,-,.,ID=KM020AG00100.cds.1;Parent=KM020AG00100
4,chr39,.,gene,4980,6154,.,-,.,ID=KM020AG00110;Name=KM020AG00110


In [48]:
gff_path = 'Klac_Kmar_synonly.gff.txt'
fasta_path = 'Klac_Kmar.fsa.txt'
orthologs_path = '~/genomes_annotations/recovery_orthofinder_FINAL_wsacc.csv'

In [8]:
#gff = pd.read_csv(gff_path, sep='\t', header=None)[[0,1,2,3,4,6,8]]
gff.columns = ['scaffold', 'source', 'type', 'start', 'stop', 'strand','info']
#gff['ID'] = gff['info'].str.split(pat=';', expand=True)
#gff['species'] = gff['ID'].str[:2]

gff.head()

,scaffold,source,type,start,stop,strand,info
0,chr39,YGOB/YGAP,gene,1721,3279,-,ID=KM020AG00100;Name=KM020AG00100
1,chr39,YGOB/YGAP,mRNA,1721,3279,-,ID=KM020AG00100;Parent=KM020AG00100;Name=KM020...
2,chr39,YGOB/YGAP,exon,1721,3279,-,ID=KM020AG00100.exon1;Parent=KM020AG00100
3,chr39,YGOB/YGAP,CDS,1721,3279,-,ID=KM020AG00100.cds.1;Parent=KM020AG00100
4,chr39,.,gene,4980,6154,-,ID=KM020AG00110;Name=KM020AG00110


### Add promoter annotations

In [43]:
promoter_size = 500

In [44]:
promoter_gff = pd.DataFrame(columns=combined_gff.columns)

for i, row in combined_gff.iterrows():
    
    promoter_row = row.copy()
    promoter_row['type'] = 'promoter'
    promoter_row['ID'] = row['ID']+'_promoter'
    
    if promoter_row['strand'] == '+':
        promoter_row['start'] = promoter_row['start'] - promoter_size
        promoter_row['stop'] = promoter_row['start']
        
    else:
        promoter_row['start'] = promoter_row['stop']
        promoter_row['stop'] = promoter_row['stop'] + promoter_size
    
    promoter_gff = promoter_gff.append(promoter_row)
    
    if i>100:
        break
    
combined_gff = combined_gff.append(promoter_gff).sort_values(['scaffold', 'start'])
combined_gff.head()

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [64]:
combined_gff['scaffold'] = combined_gff['scaffold'].apply(str)

In [65]:
combined_gff.to_csv('Klac_Kmar_CDS_and_promoters_orthologs_only.gff', sep='\t', index=None)